In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
#from sqlalchemy.ext.automap import automap_base
#from sqlalchemy.orm import Session

In [2]:
#nrows=10
Grocery_json = pd.read_json('Resources/Grocery_and_Gourmet_Food.json',lines=True, nrows=500)


In [3]:
Grocery_json

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
495,5,True,"10 13, 2015",A3AAPTA7ADEXC9,9742356831,One Fine Day,Great flavor! I purchased this after being fru...,Keep this one up your sleeve for a quick and f...,1444694400,NaN,NaN
496,5,True,"10 9, 2015",A69643EIL0ZDC,9742356831,David Miller,Good product.,Five Stars,1444348800,NaN,NaN
497,5,True,"09 28, 2015",A1T9SJTTRD2K5D,9742356831,Angela Waters,Love this curry! I used to live in Hawaii & th...,Love this curry,1443398400,NaN,NaN
498,4,True,"09 22, 2015",A1ECWZ09Y4I934,9742356831,Aijing Lin,"The taste is really great, similar to those cu...","The taste is really great, similar to those cu...",1442880000,NaN,NaN


In [4]:
Grocery_json.columns


Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image'],
      dtype='object')

In [5]:
#read labeled csv
labeled_df = pd.read_csv('Resources/labeled.csv')
labeled_df

,words,sentiment
0,"['comforting', 'know', 'processed', 'like', 's...",pos
1,"['never', 'disappointed', 'cup', 'cider', 'lik...",neg
2,"['never', 'ever', 'write', 'reviews', 'product...",neg
3,"['tasty', 'package', 'arrived', 'neatly', 'pac...",pos
4,"['gonna', 'buy', 'future', 'ate', 'completely'...",pos
...,...,...
809635,"['great', 'tasting', 'tuna', 'easy', 'open', '...",pos
809636,"['addicted', 'product', 'grocer', 'quit', 'car...",neg
809637,"['sure', 'happened', 'box', 'arrived', 'comple...",neg
809638,"['put', 'water', 'bottle', 'done', 'working', ...",pos


In [6]:
#count the number of words in a review
Grocery_json["reviewlength"] = labeled_df["words"].apply(lambda x: len(x))

Grocery_json.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,reviewlength
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,270
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,287
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,1156
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,126
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,92


In [7]:
groceries_df = Grocery_json[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92
...,...,...,...,...,...,...,...
495,9742356831,5,A3AAPTA7ADEXC9,"10 13, 2015",1444694400,Great flavor! I purchased this after being fru...,8
496,9742356831,5,A69643EIL0ZDC,"10 9, 2015",1444348800,Good product.,263
497,9742356831,5,A1T9SJTTRD2K5D,"09 28, 2015",1443398400,Love this curry! I used to live in Hawaii & th...,390
498,9742356831,4,A1ECWZ09Y4I934,"09 22, 2015",1442880000,"The taste is really great, similar to those cu...",153


In [8]:
#drop dupicates
groceries_df = groceries_df.drop_duplicates()
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92
...,...,...,...,...,...,...,...
495,9742356831,5,A3AAPTA7ADEXC9,"10 13, 2015",1444694400,Great flavor! I purchased this after being fru...,8
496,9742356831,5,A69643EIL0ZDC,"10 9, 2015",1444348800,Good product.,263
497,9742356831,5,A1T9SJTTRD2K5D,"09 28, 2015",1443398400,Love this curry! I used to live in Hawaii & th...,390
498,9742356831,4,A1ECWZ09Y4I934,"09 22, 2015",1442880000,"The taste is really great, similar to those cu...",153


In [9]:
rating_5 = groceries_df.loc[groceries_df['overall'] == 5]
rating_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
...,...,...,...,...,...,...,...
494,9742356831,5,A1TIGQEXUGWXR6,"10 13, 2015",1444694400,Love this. Moderately spicy when compared to ...,98
495,9742356831,5,A3AAPTA7ADEXC9,"10 13, 2015",1444694400,Great flavor! I purchased this after being fru...,8
496,9742356831,5,A69643EIL0ZDC,"10 9, 2015",1444348800,Good product.,263
497,9742356831,5,A1T9SJTTRD2K5D,"09 28, 2015",1443398400,Love this curry! I used to live in Hawaii & th...,390


In [10]:
rating_1 = groceries_df.loc[groceries_df['overall'] == 1]
rating_1

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97
22,4639725183,1,AE937MVR5J8OL,"12 10, 2016",1481328000,Tasted old and not individually bag,104
27,4639725183,1,A27LF8222OIRRN,"09 10, 2016",1473465600,Not the great taste everyone was talking about :(,40
37,4639725183,1,A34RH95REHOQFO,"09 25, 2015",1443139200,Terrible. What a waste of money. This is not t...,1391
69,4639725043,1,A3ELDSSPP9RZPG,"03 31, 2017",1490918400,The tea is so fine it leaves tea particles flo...,194
78,4639725043,1,A3NGPD2B16IDPY,"01 22, 2017",1485043200,This did not have the flavor expected. Definit...,332
85,4639725043,1,A1NNFPN62CFC5Q,"06 30, 2016",1467244800,foil bag not sealed!,36
91,4639725043,1,A2PMV21JFDKJIU,"03 11, 2016",1457654400,This tea looks like coffee grounds. Brewed it ...,19
113,4639725043,1,A12Z3P6W3M657S,"02 20, 2015",1424390400,"Do not buy this. Not only did expire in 2011, ...",30
131,4639725043,1,AEORSUWK2SO3R,"02 19, 2014",1392768000,Just received my brand new box of this tea fro...,448


In [11]:
rating_2 = groceries_df.loc[groceries_df['overall'] == 2]


In [12]:
rating_3 = groceries_df.loc[groceries_df['overall'] == 3]


In [13]:
rating_4 = groceries_df.loc[groceries_df['overall'] == 4]


In [14]:
groceries_df_12_45 = groceries_df[groceries_df.overall != 3]


In [15]:
#choosing every second row of the five star rating for a total of 600,000 to reduce its size from around 3.5 million ratings to a number closer to the 1 star ratings. 
rating_5_trimmed = rating_5[:1200000:2]
rating_5_trimmed

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33
...,...,...,...,...,...,...,...
489,9742356831,5,A2KXDR2MTICVNI,"10 28, 2015",1445990400,Using Thai Kitchen's red and green curry paste...,628
492,9742356831,5,A1LC9OU0CWPBRN,"10 20, 2015",1445299200,Great flavour and a good amount of heat withou...,18
494,9742356831,5,A1TIGQEXUGWXR6,"10 13, 2015",1444694400,Love this. Moderately spicy when compared to ...,98
496,9742356831,5,A69643EIL0ZDC,"10 9, 2015",1444348800,Good product.,263


In [16]:
groceries_df_1_5 = [rating_5_trimmed, rating_1]
groceries_df_1_5
groc_1_5 = pd.concat(groceries_df_1_5)
groc_1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33
...,...,...,...,...,...,...,...
351,9742356831,1,ANJMKELOZ1LKY,"05 31, 2017",1496188800,Where is my curry????? I got someone else's ma...,63
380,9742356831,1,A9X0YU2HAAI4T,"01 31, 2017",1485820800,I don't know much about authentic Thai coockin...,144
410,9742356831,1,A2FZMMOZXACZDR,"08 29, 2016",1472428800,It is too spicy and also very salty. I trashed...,131
424,9742356831,1,AS039MM100OWE,"05 30, 2016",1464566400,This doesn't taste much like green curry. The ...,84


In [17]:
g1_5 = pd.merge(rating_5_trimmed, rating_1, how='outer')
g1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
2,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
3,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257
4,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33
...,...,...,...,...,...,...,...
208,9742356831,1,ANJMKELOZ1LKY,"05 31, 2017",1496188800,Where is my curry????? I got someone else's ma...,63
209,9742356831,1,A9X0YU2HAAI4T,"01 31, 2017",1485820800,I don't know much about authentic Thai coockin...,144
210,9742356831,1,A2FZMMOZXACZDR,"08 29, 2016",1472428800,It is too spicy and also very salty. I trashed...,131
211,9742356831,1,AS039MM100OWE,"05 30, 2016",1464566400,This doesn't taste much like green curry. The ...,84


In [18]:
groceries_meta = pd.read_csv('Resources/fakemeat_meta.csv')
groceries_meta

,asin,brand,title,price
0,B000LKXRI6,Tofurky,"Turtle Island Tofurky Jurky, Peppered, 2-Ounce...",NaN
1,B000LKZL4Y,Tofurky,"Turtle Island Tofurky Jurky, Ginger Teriyaki, ...",NaN
2,B000O6I9YY,Tofurky,"Tofurky, Deli Slices, Hickory Smoked, 5.5 oz",NaN
3,B000O6K870,Boca,Boca Original Chik'n Vegan Patties Made with N...,NaN
4,B000REIDYO,Quorn,"Quorn Chicken Style Tenders, 12 Oz (Pack of 12)",NaN
...,...,...,...,...
111,B01D9DHLNS,Simply Balanced,"Simply Balanced Fruit Strips Strawberry, Apric...",$25.99
112,B01DOAQ7KE,Tofurky,"Tofurky, Deli Slices, Bologna, 5.5 oz",NaN
113,B01DOB31XY,Tofurky,"Tofurky Slow Roasted Bbq Style Chick'n, 8 Ounc...",NaN
114,B01EVO3OAS,Tofurky,"Tofurky Veggie Burger Hearty Hemp, 10.5 Ounce ...",NaN


In [19]:
groceries_meta.dtypes

asin     object
brand    object
title    object
price    object
dtype: object

In [21]:
#checked to see what the issue was with merging the data why so few rows in common. found that asin is different in the two datasets.
#groceries_meta['asin'].astype(int)

In [ ]:
#groceries_meta = groceries_meta['price'].dropna()
#groceries_meta

In [ ]:
#groceries_meta.count()

In [23]:
#groceries15merged = pd.merge(g1_5, groceries_meta, on='asin', how='outer')
#groceries15merged = [g1_5, groceries_meta]
#groceries15merged.head()
groceries15merged = [g1_5, groceries_meta]
groceries15merged
groc_1_5 = pd.concat(groceries15merged)
groc_1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength,brand,title,price
0,1888861614,5.0,ALP49FBWT4I7V,"06 4, 2013",1.370304e+09,Very pleased with my purchase. Looks exactly l...,270.0,NaN,NaN,NaN
1,1888861614,5.0,A94ZG5CWN70E7,"12 1, 2013",1.385856e+09,Omg.. The S was inexpensive and EXACTLY what w...,22.0,NaN,NaN,NaN
2,1888861614,5.0,A1B65IWLUVOUQB,"09 6, 2013",1.378426e+09,This was exactly what we were looking for to m...,44.0,NaN,NaN,NaN
3,4639725183,5.0,A1QVBUH9E1V6I8,"11 19, 2014",1.416355e+09,No adverse comment.,257.0,NaN,NaN,NaN
4,4639725183,5.0,A2CHH5U12THP2D,"11 3, 2014",1.414973e+09,Best black tea in US.\n\nHighly recommend.\nI ...,33.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
111,B01D9DHLNS,NaN,NaN,NaN,NaN,NaN,NaN,Simply Balanced,"Simply Balanced Fruit Strips Strawberry, Apric...",$25.99
112,B01DOAQ7KE,NaN,NaN,NaN,NaN,NaN,NaN,Tofurky,"Tofurky, Deli Slices, Bologna, 5.5 oz",NaN
113,B01DOB31XY,NaN,NaN,NaN,NaN,NaN,NaN,Tofurky,"Tofurky Slow Roasted Bbq Style Chick'n, 8 Ounc...",NaN
114,B01EVO3OAS,NaN,NaN,NaN,NaN,NaN,NaN,Tofurky,"Tofurky Veggie Burger Hearty Hemp, 10.5 Ounce ...",NaN


In [28]:
#groceries15merged_inner = pd.merge(g1_5, groceries_meta, on='asin', how='inner')
#groceries15merged_inner = [g1_5, groceries_meta]
#groceries15merged_inner

#groc_1_5 = pd.concat([g1_5, groceries_meta], join='inner')
#groc_1_5

,asin
0,1888861614
1,1888861614
2,1888861614
3,4639725183
4,4639725183
...,...
111,B01D9DHLNS
112,B01DOAQ7KE
113,B01DOB31XY
114,B01EVO3OAS


In [ ]:
#trying to use join to put datasets together
groc_1_5.join(other.set_index('key'), on='key')

In [30]:
#groc15merged_inner = groc_1_5[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength", 'brand', 'title']]
#groc15merged_inner

In [ ]:
groceries_csv = groc15merged_inner.to_csv(path_or_buf='Resources/groceries_merged_trimmed_FM.csv', index=False)


In [ ]:
unique_values = groc15merged_inner.brand.unique()
unique_values

In [ ]:
# only fakemeat: ratings, words in review, date, price
fakemeat_df = groc15merged_inner[['brand','title','overall','reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
fakemeat_df

In [ ]:
fakemeat_df.count()

In [ ]:
# sqlalchemy's create engine preparing the ability to connect to a sqlite database. then with .connect connecting to the database. 
engine = create_engine('sqlite:///groc15trimmed.sqlite', echo=True)
sqlite_connection = engine.connect()

In [ ]:
#connecting to database and inputting info in from dataframe
sqlite_table = "groceriesALL"
ALLgroceries=g1_5.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

In [ ]:
MLgroceries_df = g1_5[['overall', 'reviewText']]
MLgroceries_df

In [ ]:
#connecting to database and inputting info in from dataframe
sqlite_table_2 = "groceriesMLData"
MLgroceries=MLgroceries_df.to_sql(sqlite_table_2, sqlite_connection, if_exists='replace')

In [ ]:
#FM_df = fakemeat_df[['brand','overall','reviewTime','unixReviewTime', 'reviewText', 'reviewlength']]
#FM_df

In [ ]:
#connecting to database and inputting info in from dataframe
sqlite_table_3 = "fakemeatDF"
FakeMeat=fakemeat_df.to_sql(sqlite_table_3, sqlite_connection, if_exists='replace')

In [ ]:
#select and show data in groceries ALL table
#engine.execute('SELECT * FROM groceriesALL').fetchall()

In [ ]:
#engine.execute('SELECT * FROM groceriesMLData').fetchall()

In [ ]:
#engine.execute('SELECT * FROM fakemeatDF').fetchall()